In [1]:
import shutil

import dandelion as ddl
import pandas as pd

from pathlib import Path
from tqdm import tqdm

ddl.logging.print_header()

dandelion==0.3.4.dev2 pandas==2.1.0 numpy==1.24.4 matplotlib==3.7.1 networkx==3.1 scipy==1.11.2


In [2]:
samples = [
    "G000421_batch1_1",
    "G000421_batch1_2",
    "G000421_batch1_3",
    "G000421_batch1_4",
    "G000421_batch2_1",
    "G000421_batch2_2",
]

for s in tqdm(samples):
    in_path = Path("..") / "data" / "raw" / s / "vdj_b"
    out_path = Path("..") / "data" / "vdj_b" / s
    out_path.mkdir(parents=True, exist_ok=True)
    shutil.copytree(in_path, out_path, dirs_exist_ok=True)
    fasta_path = out_path / "all_contig.fasta"
    vdj = ddl.read_10x_vdj(out_path, remove_malformed=False)
    contigs = dict(vdj.data.sequence)
    ddl.utl.write_fasta(fasta_dict=contigs, out_fasta=fasta_path)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [02:12<00:00, 22.02s/it]


Once files are ready,

pull the singularity image:

```bash
DANDELIONSINGULARITY=$PATHTODANDELION/dandelion/container/sc-dandelion_latest.sif
cd ../data/vdj_b
singularity run -B $PWD $DANDELIONSINGULARITY dandelion-preprocess --meta metadata.csv --chain IG --org mouse --flavour strict --keep_trailing_hyphen_number --filter_to_high_confidence --clean_output
```